In [34]:
# This Python 3 environment comes with many helpful analytics libraries installed
# It is defined by the kaggle/python Docker image: https://github.com/kaggle/docker-python
# For example, here's several helpful packages to load

import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)

# Input data files are available in the read-only "../input/" directory
# For example, running this (by clicking run or pressing Shift+Enter) will list all files under the input directory

import os
# for dirname, _, filenames in os.walk('/kaggle/input'):
#     for filename in filenames:
#         print(os.path.join(dirname, filename))

# You can write up to 20GB to the current directory (/kaggle/working/) that gets preserved as output when you create a version using "Save & Run All" 
# You can also write temporary files to /kaggle/temp/, but they won't be saved outside of the current session

In [35]:
from torchvision.models import resnet50
import pytorch_lightning as pl
from torch.utils.data import DataLoader
from torch.utils.data import Dataset
from torchvision import transforms
from PIL import Image
import torch.cuda
import torch
import torch.nn as nn
import torch.optim as optim
from sklearn.metrics import precision_score, recall_score, f1_score, confusion_matrix
import torchvision.models as models
from pytorch_lightning.callbacks import EarlyStopping
from pytorch_lightning.callbacks import ProgressBar

In [36]:
path = "/kaggle/input/weed-dataset/weeds/weeds_dataset/"

def add_path(image_name):
    return path + image_name

def remove_missing_images(dataframe, root_dir, image_filename_column='path'):
    missing_indices = []
    for index, row in dataframe.iterrows():
        image_path = os.path.join(root_dir, row[image_filename_column])
        if not os.path.exists(image_path):
            missing_indices.append(index)
    cdf = dataframe.drop(index=missing_indices).reset_index(drop=True)
    return cdf

train0 = pd.read_csv('/kaggle/input/weed-dataset/weeds/labels/train_subset0.csv')
train1 = pd.read_csv('/kaggle/input/weed-dataset/weeds/labels/train_subset1.csv')
train2 = pd.read_csv('/kaggle/input/weed-dataset/weeds/labels/train_subset2.csv')
train3 = pd.read_csv('/kaggle/input/weed-dataset/weeds/labels/train_subset3.csv')
train4 = pd.read_csv('/kaggle/input/weed-dataset/weeds/labels/train_subset4.csv')
training_data = pd.concat([train0, train1, train2, train3, train4], ignore_index=True)

test0 = pd.read_csv('/kaggle/input/weed-dataset/weeds/labels/test_subset0.csv')
test1 = pd.read_csv('/kaggle/input/weed-dataset/weeds/labels/test_subset1.csv')
test2 = pd.read_csv('/kaggle/input/weed-dataset/weeds/labels/test_subset2.csv')
test3 = pd.read_csv('/kaggle/input/weed-dataset/weeds/labels/test_subset3.csv')
test4 = pd.read_csv('/kaggle/input/weed-dataset/weeds/labels/test_subset4.csv')
testing_data = pd.concat([test0, test1, test2, test3, test4], ignore_index=True)

val0 = pd.read_csv('/kaggle/input/weed-dataset/weeds/labels/val_subset0.csv')
val1 = pd.read_csv('/kaggle/input/weed-dataset/weeds/labels/val_subset1.csv')
val2 = pd.read_csv('/kaggle/input/weed-dataset/weeds/labels/val_subset2.csv')
val3 = pd.read_csv('/kaggle/input/weed-dataset/weeds/labels/val_subset3.csv')
val4 = pd.read_csv('/kaggle/input/weed-dataset/weeds/labels/val_subset4.csv')
validation_data = pd.concat([val0, val1, val2, val3, val4], ignore_index=True)

num_of_classes=max(train0["Label"].unique().size, testing_data["Label"].unique().size, val0["Label"].unique().size)

training_data = training_data.rename(columns={"Filename": "path"})
testing_data = testing_data.rename(columns={"Filename": "path"})
validation_data = validation_data.rename(columns={"Filename": "path"})

training_data["path"] = training_data["path"].apply(add_path)
testing_data["path"] = testing_data["path"].apply(add_path)
validation_data["path"] = validation_data["path"].apply(add_path)

training_data = remove_missing_images(training_data, path)
testing_data = remove_missing_images(testing_data, path)
validation_data = remove_missing_images(validation_data, path)

x_train = training_data["path"].tolist()
y_train = training_data["Label"].tolist()
x_val = validation_data["path"].tolist()
y_val = validation_data["Label"].tolist()
x_test = testing_data["path"].tolist()
y_test = testing_data["Label"].tolist()

In [37]:
training_data

,path,Label
0,/kaggle/input/weed-dataset/weeds/weeds_dataset...,5
1,/kaggle/input/weed-dataset/weeds/weeds_dataset...,1
2,/kaggle/input/weed-dataset/weeds/weeds_dataset...,0
3,/kaggle/input/weed-dataset/weeds/weeds_dataset...,1
4,/kaggle/input/weed-dataset/weeds/weeds_dataset...,3
...,...,...
52512,/kaggle/input/weed-dataset/weeds/weeds_dataset...,3
52513,/kaggle/input/weed-dataset/weeds/weeds_dataset...,7
52514,/kaggle/input/weed-dataset/weeds/weeds_dataset...,8
52515,/kaggle/input/weed-dataset/weeds/weeds_dataset...,8


In [38]:
training_data[training_data['Label'] == 3].count()

path     3066
Label    3066
dtype: int64

In [39]:
testing_data.Label.unique()

array([0, 8, 7, 1, 4, 3, 5, 2, 6])

In [40]:
testing_data

,path,Label
0,/kaggle/input/weed-dataset/weeds/weeds_dataset...,0
1,/kaggle/input/weed-dataset/weeds/weeds_dataset...,0
2,/kaggle/input/weed-dataset/weeds/weeds_dataset...,0
3,/kaggle/input/weed-dataset/weeds/weeds_dataset...,0
4,/kaggle/input/weed-dataset/weeds/weeds_dataset...,0
...,...,...
17502,/kaggle/input/weed-dataset/weeds/weeds_dataset...,8
17503,/kaggle/input/weed-dataset/weeds/weeds_dataset...,8
17504,/kaggle/input/weed-dataset/weeds/weeds_dataset...,8
17505,/kaggle/input/weed-dataset/weeds/weeds_dataset...,8


In [41]:
class CustomImageDataset(Dataset):
    def __init__(self, image_paths, targets, transform=None):
        self.image_paths = image_paths
        self.targets = targets
        self.transform = transform

    def __len__(self):
        return len(self.image_paths)

    def __getitem__(self, idx):
        image_path = self.image_paths[idx]
        image = Image.open(image_path).convert('RGB')
        target = self.targets[idx]

        if self.transform:
            image = self.transform(image)

        return image, target

In [ ]:
class ImageClassifier:
    def __init__(self, model, train_loader, test_loader, val_loader=None):
        self.model = model
        self.train_loader = train_loader
        self.test_loader = test_loader
        if val_loader is not None:
            self.val_loader = val_loader

    def fitt(self, epochs=1):
        callbacks = self.model.configure_callbacks()
        trainer = pl.Trainer(callbacks = callbacks, max_epochs=epochs)
        trainer.gpus=2
        if self.val_loader is not None:
            trainer.fit(self.model, self.train_loader, self.val_loader)
        trainer.fit(self.model, self.train_loader)

    def predict(self):
        self.model.eval()
        device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
        self.model.to(device)
        predictions = []
        with torch.no_grad():
            for inputs, _ in self.test_loader:
                inputs = inputs.to(device)
                outputs = self.model(inputs)
                predictions.append(outputs)
        return predictions

    def predict_proba(self):
        self.model.eval()
        device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
        self.model.to(device)
        prediction_proba = []
        softmax = nn.Softmax(dim=1)
        with torch.no_grad():
            for inputs, _ in self.test_loader:
                inputs = inputs.to(device)
                outputs = self.model(inputs)
                prediction_proba.append(softmax(outputs))
        return prediction_proba

    def predict_single(self, image_path, transform):
        self.model.eval()
        device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
        self.model.to(device)
        
        image = Image.open(image_path).convert('RGB')

        if transform:
            image = transform(image).unsqueeze(0).to(device)
        
        with torch.no_grad():
            output = self.model(image)
        
        return output

    def predict_single_proba(self, image_path, transform):
        self.model.eval()
        device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
        self.model.to(device)
        
        image = Image.open(image_path).convert('RGB')

        if transform:
            image = transform(image).unsqueeze(0).to(device)
        
        with torch.no_grad():
            output = self.model(image)
            proba = nn.Softmax(dim=1)(output)
        
        return proba
    
    def Confussion_matrix(self):
        self.model.eval()
        device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
        self.model.to(device)
        correct = 0
        total = 0
        predicted_labels = []
        true_labels = []

        with torch.no_grad():
            for inputs, targets in self.test_loader:
                inputs = inputs.to(device)
                targets = targets.to(device)
                outputs = self.model(inputs)
                _, predicted = torch.max(outputs, 1)
                total += targets.size(0)
                correct += (predicted == targets).sum().item()
                predicted_labels.extend(predicted.cpu().numpy())
                true_labels.extend(targets.cpu().numpy())

        accuracy = 100 * correct / total
        print(f'Accuracy on test data: {accuracy:.2f}%')

        precision = precision_score(true_labels, predicted_labels, average='macro')
        recall = recall_score(true_labels, predicted_labels, average='macro')
        f1 = f1_score(true_labels, predicted_labels, average='macro')

        print('Precision: {:.4f}'.format(precision))
        print('Recall: {:.4f}'.format(recall))
        print('F1 Score: {:.4f}'.format(f1))

        conf_matrix = confusion_matrix(true_labels, predicted_labels)
        print('Confusion Matrix:')
        print(conf_matrix)
        
    def save_model(self, save_path):
        torch.save(RestNet_50_model.state_dict(), save_path)
        

In [42]:
class ResNetClassifier(pl.LightningModule):
    def __init__(self, learning_rate=0.01, num_of_classes=9, y_train=y_train):
        super(ResNetClassifier, self).__init__()
        self.model = resnet50(pretrained=False)
        num_ftrs = self.model.fc.in_features
        self.model.fc = nn.Linear(num_ftrs, num_of_classes)
        self.learning_rate = learning_rate

        class_sample_count = np.array([sum(y_train == t) for t in np.unique(y_train)])
        weight = 1. / class_sample_count
        samples_weight = np.array([weight[t] for t in y_train])
        self.class_weights = torch.FloatTensor(weight).to(self.device)

        self.criterion = nn.CrossEntropyLoss(weight=self.class_weights)

    def forward(self, x):
        return self.model(x)

    def training_step(self, batch, batch_idx):
        inputs, labels = batch
        outputs = self(inputs)
        loss = self.criterion(outputs, labels)
        self.log('train_loss', loss)
        return loss

    def configure_optimizers(self):
        optimizer = optim.SGD(self.parameters(), lr=self.learning_rate, momentum=0.9)
        return optimizer

    def on_train_epoch_end(self):
        torch.cuda.empty_cache()

In [43]:
def image_transform(image):
    transform = transforms.Compose([
        transforms.Resize((224, 224)),
        transforms.ToTensor(),
        transforms.Normalize(mean=[0.485, 0.456, 0.406], std=[0.229, 0.224, 0.225])
    ])
    return transform(image)

In [44]:
train_dataset_Restnet_50 = CustomImageDataset(x_train, y_train, transform=image_transform)
test_dataset_Restnet_50 = CustomImageDataset(x_test, y_test, transform=image_transform)

train_loader_Restnet_50 = DataLoader(train_dataset_Restnet_50, batch_size=32, shuffle=True)
test_loader_Restnet_50 = DataLoader(test_dataset_Restnet_50, batch_size=32, shuffle=False)

In [45]:
RestNet_50 = ResNetClassifier()
print(RestNet_50)

/opt/conda/lib/python3.10/site-packages/torchvision/models/_utils.py:208: UserWarning: The parameter 'pretrained' is deprecated since 0.13 and may be removed in the future, please use 'weights' instead.
  warnings.warn(
/opt/conda/lib/python3.10/site-packages/torchvision/models/_utils.py:223: UserWarning: Arguments other than a weight enum or `None` for 'weights' are deprecated since 0.13 and may be removed in the future. The current behavior is equivalent to passing `weights=None`.
  warnings.warn(msg)


ResNetClassifier(
  (model): ResNet(
    (conv1): Conv2d(3, 64, kernel_size=(7, 7), stride=(2, 2), padding=(3, 3), bias=False)
    (bn1): BatchNorm2d(64, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
    (relu): ReLU(inplace=True)
    (maxpool): MaxPool2d(kernel_size=3, stride=2, padding=1, dilation=1, ceil_mode=False)
    (layer1): Sequential(
      (0): Bottleneck(
        (conv1): Conv2d(64, 64, kernel_size=(1, 1), stride=(1, 1), bias=False)
        (bn1): BatchNorm2d(64, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
        (conv2): Conv2d(64, 64, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1), bias=False)
        (bn2): BatchNorm2d(64, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
        (conv3): Conv2d(64, 256, kernel_size=(1, 1), stride=(1, 1), bias=False)
        (bn3): BatchNorm2d(256, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
        (relu): ReLU(inplace=True)
        (downsample): Sequential(
   

In [ ]:
RestNet_50_model = ImageClassifier(RestNet_50, train_loader_Restnet_50, test_dataset_Restnet_50)

In [46]:
RestNet_50_model.fitt(epochs=50)

In [ ]:
RestNet_50_model.Confussion_matrix()

In [ ]:
save_path = '/kaggle/working/my_model_Restnet_50.pth'
RestNet_50_model.save_model(save_path)

In [47]:
class InceptionV3Classifier(pl.LightningModule):
    def __init__(self, learning_rate=0.01, num_of_classes=9, y_train=None):
        super(InceptionV3Classifier, self).__init__()
        self.model = models.inception_v3(pretrained=False)
        
        num_ftrs = self.model.fc.in_features
        self.model.fc = nn.Linear(num_ftrs, num_of_classes)
        
        num_aux_ftrs = self.model.AuxLogits.fc.in_features
        self.model.AuxLogits.fc = nn.Linear(num_aux_ftrs, num_of_classes)
        
        self.learning_rate = learning_rate
        
        if y_train is not None:
            class_sample_count = np.array([sum(y_train == t) for t in np.unique(y_train)])
            weight = 1. / class_sample_count
            self.class_weights = torch.FloatTensor(weight).to(self.device)
        else:
            self.class_weights = None

        self.criterion = nn.CrossEntropyLoss(weight=self.class_weights)

    def forward(self, x):
        return self.model(x)

    def training_step(self, batch, batch_idx):
        inputs, labels = batch
        inputs, labels = inputs.to(self.device), labels.to(self.device)
        outputs = self(inputs)

        # The InceptionV3 model returns a tuple of (main_output, aux_output)
        main_output, aux_output = outputs.logits, outputs.aux_logits

        loss1 = self.criterion(main_output, labels)
        loss2 = self.criterion(aux_output, labels)
        loss = loss1 + 0.4 * loss2

        self.log('train_loss', loss)
        return loss

    def configure_optimizers(self):
        optimizer = optim.SGD(self.parameters(), lr=self.learning_rate, momentum=0.9)
        return optimizer

    def on_train_epoch_end(self):
        torch.cuda.empty_cache()


In [48]:
def image_transform1(image):
    transform = transforms.Compose([
        transforms.Resize((299, 299)),
        transforms.ToTensor(),
        transforms.Normalize(mean=[0.485, 0.456, 0.406], std=[0.229, 0.224, 0.225])
    ])
    return transform(image)

In [25]:
train_dataset_InceptionV3 = CustomImageDataset(x_train, y_train, transform=image_transform1)
test_dataset_InceptionV3 = CustomImageDataset(x_test, y_test, transform=image_transform1)

train_loader_InceptionV3 = DataLoader(train_dataset_InceptionV3, batch_size=16, shuffle=True)
test_loader_InceptionV3 = DataLoader(test_dataset_InceptionV3, batch_size=16, shuffle=False)

In [49]:
inception_model = InceptionV3Classifier(y_train=y_train)

/opt/conda/lib/python3.10/site-packages/torchvision/models/inception.py:43: FutureWarning: The default weight initialization of inception_v3 will be changed in future releases of torchvision. If you wish to keep the old behavior (which leads to long initialization times due to scipy/scipy#11299), please set init_weights=True.
  warnings.warn(


In [ ]:
inception_V3_model = ImageClassifier(inception_model, train_loader_InceptionV3, test_loader_InceptionV3)

In [ ]:
inception_V3_model.fitt(epochs = 50)

In [ ]:
inception_V3_model.Confussion_matrix()

In [ ]:
save_path = '/kaggle/working/my_model_inception_v3.pth'
inception_V3_model.save_model(save_path)

In [50]:
class AlexNet(pl.LightningModule):
    def __init__(self, num_classes=9, y_train=y_train):
        super(AlexNet, self).__init__()
        self.features = nn.Sequential(
            nn.Conv2d(3, 96, kernel_size=11, stride=4),
            nn.ReLU(inplace=True),
            nn.MaxPool2d(kernel_size=3, stride=2),
            nn.Conv2d(96, 256, kernel_size=5, padding=2),
            nn.ReLU(inplace=True),
            nn.MaxPool2d(kernel_size=3, stride=2),
            nn.Conv2d(256, 384, kernel_size=3, padding=1),
            nn.ReLU(inplace=True),
            nn.Conv2d(384, 384, kernel_size=3, padding=1),
            nn.ReLU(inplace=True),
            nn.Conv2d(384, 256, kernel_size=3, padding=1),
            nn.ReLU(inplace=True),
            nn.MaxPool2d(kernel_size=3, stride=2),
        )
        self.avgpool = nn.AdaptiveAvgPool2d((6, 6))
        self.classifier = nn.Sequential(
            nn.Dropout(),
            nn.Linear(256 * 6 * 6, 4096),
            nn.ReLU(inplace=True),
            nn.Dropout(),
            nn.Linear(4096, 4096),
            nn.ReLU(inplace=True),
            nn.Linear(4096, num_classes),
        )
        
        if y_train is not None:
            y_train = torch.tensor(y_train)
            class_sample_count = torch.tensor([(y_train == t).sum() for t in torch.unique(y_train)])
            weight = 1. / class_sample_count.float()
            self.class_weights = torch.FloatTensor(weight).to('cuda')
        else:
            self.class_weights = None
        
        self.criterion = nn.CrossEntropyLoss(weight=self.class_weights)

    def forward(self, x):
        x = self.features(x)
        x = self.avgpool(x)
        x = torch.flatten(x, 1)
        x = self.classifier(x)
        return x

    def training_step(self, batch, batch_idx):
        x, y = batch
        logits = self(x)
        loss = self.criterion(logits, y)
        self.log('train_loss', loss)
        return loss

    def configure_optimizers(self):
        return optim.SGD(self.parameters(), lr=0.001)
    
    def on_train_epoch_end(self):
        torch.cuda.empty_cache()

In [51]:
def image_transform2(image):
    transform = transforms.Compose([
        transforms.RandomResizedCrop(224),
        transforms.RandomHorizontalFlip(),
        transforms.ToTensor(),
        transforms.Normalize(mean=[0.485, 0.456, 0.406], std=[0.229, 0.224, 0.225]),
    ])
    return transform(image)

In [52]:
train_dataset_AlexNet = CustomImageDataset(x_train, y_train, transform=image_transform2)
test_dataset_AlexNet = CustomImageDataset(x_test, y_test, transform=image_transform2)

train_loader_AlexNet = DataLoader(train_dataset_AlexNet, batch_size=16, shuffle=True)
test_loader_AlexNet = DataLoader(test_dataset_AlexNet, batch_size=16, shuffle=False)

In [53]:
Alexnet_model = AlexNet()

In [ ]:
AlexNet_model = ImageClassifier(Alexnet_model, train_loader_AlexNet, test_loader_AlexNet)

In [ ]:
trainer.fitt(epochs = 50)

In [ ]:
AlexNet_model.Confussion_matrix()

In [ ]:
save_path = '/kaggle/working/my_model_AlexNet_model.pth'
AlexNet_model.save_model(save_path)

In [63]:
saved_model_path_resnet = '/kaggle/input/restnet50model/pytorch/resnet/1/Restnet_50.pth'
saved_model_path_inception = '/kaggle/input/inceptionv3/pytorch/model_1/1/my_model_inception_v3.pth'
saved_model_path_alexnet = '/kaggle/input/alexnetwork/pytorch/model/1/AlexNet_model.pth'

ResNet_50_model_saved = ResNetClassifier(y_train=y_train)
ResNet_50_model_saved.load_state_dict(torch.load(saved_model_path_resnet))

model_inception_v3_saved = InceptionV3Classifier(y_train=y_train)
model_inception_v3_saved.load_state_dict(torch.load(saved_model_path_inception))

model_alexnet_saved = AlexNet(y_train=y_train)
model_alexnet_saved.load_state_dict(torch.load(saved_model_path_alexnet))


/opt/conda/lib/python3.10/site-packages/torchvision/models/inception.py:43: FutureWarning: The default weight initialization of inception_v3 will be changed in future releases of torchvision. If you wish to keep the old behavior (which leads to long initialization times due to scipy/scipy#11299), please set init_weights=True.
  warnings.warn(


<All keys matched successfully>

In [ ]:
def image_transform3(image):
    transform = transforms.Compose([
        transforms.Resize((299, 299)),
        transforms.RandomHorizontalFlip(),
        transforms.ToTensor(),
        transforms.Normalize(mean=[0.485, 0.456, 0.406], std=[0.229, 0.224, 0.225]),
    ])
    return transform(image)

In [ ]:
train_dataset_stacking = CustomImageDataset(x_train, y_train, transform=image_transform3)
test_dataset_stacking = CustomImageDataset(x_test, y_test, transform=image_transform3)
val_dataset_stacking = CustomImageDataset(x_val, y_val, transform=image_transform3)

train_loader_stacking = DataLoader(train_dataset_stacking, batch_size=16, shuffle=True)
test_loader_stacking = DataLoader(test_dataset_stacking, batch_size=16, shuffle=False)
val_loader_stacking = DataLoader(val_dataset_stacking, batch_size=16, shuffle=False)

In [ ]:
class StackingMetaModel(pl.LightningModule):
    def __init__(self, resnet_model = ResNet_50_model_saved, inception_model = model_inception_v3_saved, alexnet_model = model_alexnet_saved, num_classes=9, y_train=y_train):
        super(StackingMetaModel, self).__init__()
        self.resnet_model = resnet_model
        self.inception_model = inception_model
        self.alexnet_model = alexnet_model

        self.meta_classifier = nn.Linear(3 * num_classes, num_classes)

        self.val_losses = []
        self.validation_step_outputs = []
        self.training_step_outputs = []
        
        if y_train is not None:
            y_train = torch.tensor(y_train)
            class_sample_count = torch.tensor([(y_train == t).sum() for t in torch.unique(y_train)])
            weight = 1. / class_sample_count.float()
            self.class_weights = torch.FloatTensor(weight).to('cuda')
        else:
            self.class_weights = None
        
        self.criterion = nn.CrossEntropyLoss(weight=self.class_weights)

        self.early_stopping = EarlyStopping(monitor="val_loss", patience=3, mode="min")

    def forward(self, x):
        resnet_output = self.resnet_model(x)
        inception_output = self.inception_model(x)
        alexnet_output = self.alexnet_model(x)

        if hasattr(inception_output, 'logits'):
            inception_output = inception_output.logits

        stacked_output = torch.cat((resnet_output, inception_output, alexnet_output), dim=1)
        return self.meta_classifier(stacked_output)

    def training_step(self, batch, batch_idx):
        x, y = batch
        outputs = self(x)
        loss = self.criterion(outputs, y)
        self.log('train_loss', loss)
        self.training_step_outputs.append(loss)
        return loss

    def configure_optimizers(self):
        optimizer = optim.SGD(self.parameters(), lr=0.001, momentum=0.9)
        return optimizer

    def validation_step(self, batch, batch_idx):
        x, y = batch
        outputs = self(x)
        loss = self.criterion(outputs, y)
        self.log('val_loss', loss)
        self.validation_step_outputs.append(loss)

    def on_validation_epoch_end(self):
        avg_loss = torch.stack(self.validation_step_outputs).mean()
        self.log('avg_val_loss', avg_loss)
        self.validation_step_outputs.clear()

    def on_train_epoch_end(self):
        avg_loss = torch.stack(self.training_step_outputs).mean()
        self.log('avg_train_loss', avg_loss)
        self.training_step_outputs.clear()

    def configure_callbacks(self):
        return [self.early_stopping]

In [ ]:
stacking_meta_model = StackingMetaModel()
Stacking_Meta_Model = ImageClassifier(stacking_meta_model, train_loader_stacking, test_loader_stacking, val_loader_stacking)

In [ ]:
Stacking_Meta_Model.fitt(epochs = 50)

In [ ]:
Stacking_Meta_Model.Confussion_matrix()

In [ ]:
save_path = '/kaggle/working/my_model_Stacking.pth'
Stacking_Meta_Model.save_model(save_path)

In [ ]:
train_dataset_voting = CustomImageDataset(x_train, y_train, transform=image_transform3)
test_dataset_voting = CustomImageDataset(x_test, y_test, transform=image_transform3)
val_dataset_voting = CustomImageDataset(x_val, y_val, transform=image_transform3)

train_loader_voting = DataLoader(train_dataset_voting, batch_size=32, shuffle=True)
test_loader_voting = DataLoader(test_dataset_voting, batch_size=32, shuffle=False)
val_loader_voting = DataLoader(val_dataset_voting, batch_size=32, shuffle=False)

In [ ]:
class VotingMetaModel(pl.LightningModule):
    def __init__(self, resnet_model=ResNet_50_model_saved, inception_model=model_inception_v3_saved, alexnet_model=model_alexnet_saved, num_classes=9):
        super(VotingMetaModel, self).__init__()
        self.resnet_model = resnet_model
        self.inception_model = inception_model
        self.alexnet_model = alexnet_model

        self.val_losses = []
        self.validation_step_outputs = []
        self.training_step_outputs = []

        self.early_stopping = EarlyStopping(monitor="val_loss", patience=3, mode="min")

    def forward(self, x):
        resnet_output = self.resnet_model(x)
        inception_output = self.inception_model(x)
        alexnet_output = self.alexnet_model(x)

        if hasattr(inception_output, 'logits'):
            inception_output = inception_output.logits

        outputs = [resnet_output, inception_output, alexnet_output]

        avg_output = torch.mean(torch.stack(outputs), dim=0)
        return avg_output

    def training_step(self, batch, batch_idx):
        x, y = batch
        outputs = self(x)
        loss = nn.functional.cross_entropy(outputs, y)
        self.log('train_loss', loss)
        self.training_step_outputs.append(loss)
        return loss

    def configure_optimizers(self):
        optimizer = optim.SGD(self.parameters(), lr=0.001, momentum=0.9)
        return optimizer

    def validation_step(self, batch, batch_idx):
        x, y = batch
        outputs = self(x)
        loss = nn.functional.cross_entropy(outputs, y)
        self.log('val_loss', loss)
        self.validation_step_outputs.append(loss)

    def on_validation_epoch_end(self):
        avg_loss = torch.stack(self.validation_step_outputs).mean()
        self.log('avg_val_loss', avg_loss)
        self.validation_step_outputs.clear()

    def on_train_epoch_end(self):
        avg_loss = torch.stack(self.training_step_outputs).mean()
        self.log('avg_train_loss', avg_loss)
        self.training_step_outputs.clear()

    def configure_callbacks(self):
        return [self.early_stopping]

In [ ]:
voting_meta_model_soft = VotingMetaModel()

In [ ]:
Voting_Meta_Model = ImageClassifier(voing_meta_model_soft, train_loader_voting, test_loader_voting, val_loader_voting)

In [ ]:
Voting_Meta_Model.fitt(epochs = 50)

In [ ]:
Voting_Meta_Model.Confussion_matrix()